In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd 

from tqdm import tqdm
from utils import *

In [3]:
train_set, train_labels, test_set, test_labels = load_and_preprocess_data(path='./data/CIFAR10')

Files already downloaded and verified
Files already downloaded and verified


In [4]:
mu = 1e-3
rho = 1e-4

input_size = 32 * 32 * 3
hidden_size = 2048
output_size = 10

In [5]:
def train_network(model: NeuralNetwork, train_set, train_labels, num_epochs):
    N_samples = train_set.shape[0]

    # Add accuracy and loss tracking

    for epoch in range(num_epochs):
        
        data_idx = np.random.permutation(N_samples)
        running_epoch_loss = 0
        running_predictions = []
        running_labels = []
        
        pbar = tqdm(total=N_samples)
        for iter, idx in enumerate(data_idx):
            h = train_set[idx]
            gamma = transform_label(train_labels[idx])

            gamma_hat = model.forward(h)
            loss = cross_entropy_loss(gamma_hat, gamma, model.rho, model.weights())
            model.backward(gamma_hat, gamma)

            running_epoch_loss += loss
    
            running_predictions.append(np.argmax(gamma_hat))
            running_labels.append(train_labels[idx])

            pbar.update(1)
            pbar.set_description(f'Train loss: {loss:.4f}')
        pbar.close()

        epoch_loss = running_epoch_loss / N_samples
        epoch_accuracy = np.mean(np.array(running_predictions) == np.array(running_labels))

        print(f"Epoch: {epoch + 1} | Train Loss {epoch_loss:.4f} | Train Accuracy: {epoch_accuracy * 100:.2f}%")

    model.normalize_weights()

In [6]:
def evaluate(model: NeuralNetwork, dataset, labels):
    N_samples = dataset.shape[0]

    running_predictions = []
    running_labels = []

    pbar = tqdm(total=N_samples)
    for idx in range(N_samples):
        h = dataset[idx]
        gamma = transform_label(labels[idx])
        gamma_hat = model.forward(h)

        running_predictions.append(np.argmax(gamma_hat))
        running_labels.append(labels[idx])

        pbar.update(1)
        pbar.set_description("Evaluating...")
    pbar.close()

    num_misclassified = np.sum(np.array(running_predictions) != np.array(running_labels))
    error = num_misclassified / N_samples
    accuracy = 1 - error

    print(f"Accuracy: {accuracy * 100:.2f}% | Error: {error * 100:.2f}% | Misclassified: {num_misclassified}")

    return accuracy, error, num_misclassified


### Training without dropout

In [7]:
model = NeuralNetwork(input_size, hidden_size, output_size, ReLU, d_ReLU, mu, rho)

In [8]:
train_network(model, train_set[:5000], train_labels[:5000], num_epochs=10)

Train loss: 3.2180: 100%|██████████| 5000/5000 [05:04<00:00, 16.43it/s]


Epoch: 1 | Train Loss 2.1334 | Train Accuracy: 27.84%


Train loss: 3.8849: 100%|██████████| 5000/5000 [05:03<00:00, 16.47it/s]


Epoch: 2 | Train Loss 1.8486 | Train Accuracy: 38.80%


Train loss: 3.8922: 100%|██████████| 5000/5000 [05:03<00:00, 16.48it/s]


...

Epoch: 8 | Train Loss 1.1911 | Train Accuracy: 62.90%


Train loss: 1.7069: 100%|██████████| 5000/5000 [04:56<00:00, 16.86it/s]


Epoch: 9 | Train Loss 1.0878 | Train Accuracy: 67.00%


Train loss: 0.5474: 100%|██████████| 5000/5000 [05:15<00:00, 15.83it/s]

Epoch: 10 | Train Loss 0.9865 | Train Accuracy: 69.70%


In [9]:
train_acc, train_err, train_misc = evaluate(model, train_set[:5000], train_labels[:5000])

Evaluating...: 100%|██████████| 5000/5000 [00:48<00:00, 103.26it/s]

Accuracy: 74.14% | Error: 25.86% | Misclassified: 1293


In [10]:
test_acc, test_err, test_misc = evaluate(model, test_set, test_labels)

Evaluating...: 100%|██████████| 10000/10000 [01:30<00:00, 110.27it/s]

Accuracy: 42.00% | Error: 58.00% | Misclassified: 5800


### Training with dropout

In [11]:
model = NeuralNetwork(input_size, hidden_size, output_size, ReLU, d_ReLU, mu, rho, p_dropout=[0.1, 0.5, 0.5])

In [12]:
train_network(model, train_set[:5000], train_labels[:5000], num_epochs=15)

Train loss: 2.4097: 100%|██████████| 5000/5000 [04:44<00:00, 17.60it/s]


Epoch: 1 | Train Loss 2.3833 | Train Accuracy: 17.00%


Train loss: 1.6695: 100%|██████████| 5000/5000 [04:46<00:00, 17.47it/s]


Epoch: 2 | Train Loss 2.1542 | Train Accuracy: 25.02%


Train loss: 1.9472: 100%|██████████| 5000/5000 [04:49<00:00, 17.27it/s]


...

Epoch: 13 | Train Loss 1.6554 | Train Accuracy: 44.68%


Train loss: 0.4665: 100%|██████████| 5000/5000 [04:54<00:00, 17.01it/s]


Epoch: 14 | Train Loss 1.6275 | Train Accuracy: 46.46%


Train loss: 0.8303: 100%|██████████| 5000/5000 [05:04<00:00, 16.43it/s]

Epoch: 15 | Train Loss 1.5969 | Train Accuracy: 47.50%


In [13]:
train_acc_do, train_err_do, train_misc_do = evaluate(model, train_set[:5000], train_labels[:5000])

Evaluating...: 100%|██████████| 5000/5000 [00:34<00:00, 143.54it/s]

Accuracy: 41.06% | Error: 58.94% | Misclassified: 2947


In [14]:
test_acc_do, test_err_do, test_misc_do = evaluate(model, test_set, test_labels)

Evaluating...: 100%|██████████| 10000/10000 [01:07<00:00, 147.88it/s]

Accuracy: 33.81% | Error: 66.19% | Misclassified: 6619


### Tables

In [16]:
pd.DataFrame([
    {
        "Setting": "w/o Dropout",
        "Test Error [%]": test_err*100,
        "# Test Misclassified": test_misc,
        "Train Error [%]": train_err*100,
        "# Train Misclassified": train_misc,
    },
    {
        "Setting": "w/ Dropout",
        "Test Error [%]": test_err_do*100,
        "# Test Misclassified": test_misc_do,
        "Train Error [%]": train_err_do*100,
        "# Train Misclassified": train_misc_do,
    }

])

,Setting,Test Error [%],# Test Misclassified,Train Error [%],# Train Misclassified
0,w/o Dropout,58.00,5800,25.86,1293
1,w/ Dropout,66.19,6619,58.94,2947
